In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.tri as MTri

In [11]:
# Optimization Parameters
ConstitutiveModels = ['Ogden', 'NeoHookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Real']
BCsType  = BCsTypes[1] + ' Boundary Conditions'

Tissues = ['Cortex', 'Basal Ganglia', 'Corona Radiata', 'Corpus Callosum']
Tissue = Tissues[3]

NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

# Folder for the results
Optim_Path = os.path.join('../Optimization/Results', Tissue, ConstitutiveModel, BCsType)

OptiName = str(NumberElements) + 'Elements.txt'
MarcName = str(NumberElements) + 'Elements-M.txt'

# Cost Function Data
CostData = pd.read_csv('Data/Grid ' + Tissue + '.csv', sep=',', decimal='.')
OptiData = pd.read_csv(os.path.join(Optim_Path, OptiName), sep=' ', decimal='.')

# Row with the minimum cost
print('Min cost point for:')
print(CostData.loc[CostData['TotalCostNormMax'].idxmin()])
print('Optimization starting point:')
print(OptiData.loc[0])
print('Optimization results:')
print(OptiData.iloc[-1])

Subsampling = 1

Lambdas = CostData.Lambda
Mus     = CostData.Mu
Alphas  = CostData.Alpha
Costs   = CostData.TotalCostNormMax

# # 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
CostImage = ax.scatter(Lambdas[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c=Costs, marker = '.', label='Points tested')
Colorbar = plt.colorbar(CostImage)
Colorbar.set_label('Cost')
ax.set_xlabel('Lambda (kPa)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.legend()
plt.savefig('Plots/' + Tissue + 'CompleteGrid.png')
plt.show()

Min cost point for:
Alpha                             -33
BCsType                         Fixed
CompressionCostNoNorm      0.00101312
CompressionCostNormMax     0.00519688
CompressionCostStep           58.7203
ConstitutiveModel               Ogden
Lambda                            1.4
Mu                                0.3
NumberElements                     20
SimpleShearCostNoNorm     0.000181923
SimpleShearCostNormMax     0.00900712
SimpleShearCostStep          0.948116
TensileCostNoNorm         7.12789e-05
TensileCostNormMax         0.00959112
TensileCostStep               60.2577
TotalCostNoNorm           0.000422107
TotalCostNormMax            0.0079317
TotalCostStep                 39.9754
Name: 6560, dtype: object
Optimization starting point:
Lambda                10.000
Mu                     0.350
Alpha                -25.300
CompressionNormMax     1.557
TensionNormMax        15.720
SimpleShearNormMax     0.055
TotalCostNormMax       5.777
Name: 0, dtype: float64
Optimization r

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Filtered 3D Plot

FilteredRangeValues = [0,0.1]

Filter1 = CostData['TotalCostNormMax'] > FilteredRangeValues[0]
Filter2 = CostData['TotalCostNormMax'] < FilteredRangeValues[1]

FilteredCost = CostData[Filter1 & Filter2]

%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Lambdas = FilteredCost.Lambda
Mus     = FilteredCost.Mu
Alphas  = FilteredCost.Alpha
Costs   = FilteredCost['TotalCostNormMax']

CostImage = ax.scatter(Lambdas, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Lambda (kPa)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Lambdas[FilteredCost['TotalCostNormMax'].idxmin()]], [Mus[FilteredCost['TotalCostNormMax'].idxmin()]], [Alphas[FilteredCost['TotalCostNormMax'].idxmin()]], marker = 'o', color='k')
ax.plot([OptiData.Lambda[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r')
ax.plot([OptiData.Lambda[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m')
ax.plot(OptiData.Lambda, OptiData.Mu, OptiData.Alpha, marker = None, color='m')
# Build the legend
ax.plot([],[], marker = 'o', linestyle='none', color='m', label='Budday starting point')
ax.plot([],[], color='m', label='Optimization path')
ax.plot([],[], marker = 'o', linestyle='none', color='r', label='Optimization result')
ax.plot([],[], marker = '.', linestyle='none', color='y', label='Points tested')
ax.plot([],[], marker = 'o', linestyle='none', color='k', label='Grid min cost point')
ax.legend()
plt.savefig('Plots/' + Tissue + 'FilteredGrid.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
MinPoint = CostData['TotalCostNormMax'].idxmin()

AxisNames = ['Lambda', 'Mu', 'Alpha']
Units = [' (kPa)', ' (kPa)', ' (-)']

%matplotlib widget
Figure, Axes  = plt.subplots(1,3, figsize=(12,4))

for Projection in range(len(AxisNames)):
    
    # Filter data to get the correct slice
    SliceAxis = CostData[AxisNames[Projection]].loc[MinPoint]
    Filter = CostData[AxisNames[Projection]] == SliceAxis
    Slice = CostData[Filter]
    
    # Data for plot
    XAxis = AxisNames[Projection-2]
    YAxis = AxisNames[Projection-1]
    
    XData = Slice[XAxis]
    YData = Slice[YAxis]
    Color = Slice['TotalCostNormMax'].values
    
    Table = Slice.pivot(XAxis,YAxis,'TotalCostNormMax')
    XPositions = Table.index.values
    YPositions = Table.columns.values
    Contourlevels = Table.transpose().values
    
    # Levels for contour
    SortedSlice = Slice.sort_values('TotalCostNormMax')
    SampledLevels = SortedSlice['TotalCostNormMax'][0::100].values
    Vmax = SampledLevels[-1]
    SampledLevels = np.append(SampledLevels,Slice['TotalCostNormMax'].max())
    
    # Plot
    Axes[Projection].scatter(XData,YData,c=Color,marker='.',s=10,vmax=Vmax)
    Axes[Projection].contour(XPositions,YPositions,Contourlevels,levels=SampledLevels,vmax=Vmax)
    CostContour = Axes[Projection].contourf(XPositions,YPositions,Contourlevels,levels=SampledLevels,vmax=Vmax,alpha=0.85)
    
    Axes[Projection].plot(CostData[XAxis][MinPoint], CostData[YAxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
    Axes[Projection].plot(OptiData[XAxis][len(OptiData)-1], OptiData[YAxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)
    
    
    # Plot Settings
    Axes[Projection].set_xlabel(XAxis+Units[Projection-2])
    Axes[Projection].set_ylabel(YAxis+Units[Projection-1])
    
    ColorBar = Figure.colorbar(CostContour,ax=Axes[Projection], format='%0.2f')
    ColorBar.set_label('Cost (-)')
    
    if Projection+1 == len(AxisNames):
        plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
        plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData['TotalCostNormMax'].min()))
        plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCostNormMax'][len(OptiData)-1])
        Figure.legend(loc='lower center', framealpha=1, ncol=3)
        plt.subplots_adjust(left=0.075, bottom=0.25, right=0.95, top=0.95, wspace=0.5, hspace=None)
        
plt.savefig('Plots/' + Tissue + 'MinPointProjection.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
%matplotlib widget

Figure, Axis = plt.subplots(1, 1, figsize=(10, 7))
Axis.remove()
Axis = Figure.add_subplot(1,1,1, projection='3d')

AxisNames = ['Lambda', 'Mu', 'Alpha']
Units = [' (kPa)', ' (kPa)', ' (-)']
SliceAxes = ['x','y','z']

MinPoint = CostData['TotalCostNormMax'].idxmin()
x0 = CostData.loc[MinPoint].Lambda
y0 = CostData.loc[MinPoint].Mu
z0 = CostData.loc[MinPoint].Alpha

def f(x=x0, y=y0, z=z0):
        
    XAxis, YAxis, ZAxis = AxisNames
        
    # Filters
    Filter1 = CostData[XAxis] == x
    Filter2 = CostData[YAxis] == y
    Filter3 = CostData[ZAxis] == z
        
    Filters = [Filter1,Filter2,Filter3]
    
    Point = CostData[Filter1&Filter2&Filter3]
    
    Axis.clear()
    
    for Projection in range(len(Filters)):
        
        Slice = CostData[Filters[Projection]]

        # Data for plot
        Axis1 = AxisNames[Projection-2]
        Axis2 = AxisNames[Projection-1]
        Axis3 = AxisNames[Projection]

        Data1 = Slice[Axis1]
        Data2 = Slice[Axis2]
        Color = Slice['TotalCostNormMax'].values

        Table = Slice.pivot(Axis1,Axis2,'TotalCostNormMax')
        Positions1 = Table.index.values
        Positions2 = Table.columns.values
        Contourlevels = Table.transpose().values

        Positions1, Positions2 = np.meshgrid(Positions1,Positions2)

        # Levels for contour
        SortedSlice = Slice.sort_values('TotalCostNormMax')
        SampledLevels = SortedSlice['TotalCostNormMax'][0::100].values
        Vmax = SampledLevels[-1]
        SampledLevels = np.append(SampledLevels,Slice['TotalCostNormMax'].max())

        # Plot contour
        if Projection == 0:
            Axis.contour(Contourlevels,Positions1,Positions2,zdir=SliceAxes[Projection],offset=CostData[Axis3].min(),levels=SampledLevels,vmax=Vmax)
            Axis.contourf(Contourlevels,Positions1,Positions2,zdir=SliceAxes[Projection],offset=CostData[Axis3].min(),levels=SampledLevels,vmax=Vmax,alpha=0.2)
        elif Projection == 1:
            Axis.contour(Positions2,Contourlevels,Positions1,zdir=SliceAxes[Projection],offset=CostData[Axis3].max(),levels=SampledLevels,vmax=Vmax)
            Axis.contourf(Positions2,Contourlevels,Positions1,zdir=SliceAxes[Projection],offset=CostData[Axis3].max(),levels=SampledLevels,vmax=Vmax,alpha=0.2)
        elif Projection == 2:
            Axis.contour(Positions1,Positions2,Contourlevels,zdir=SliceAxes[Projection],offset=CostData[Axis3].min(),levels=SampledLevels,vmax=Vmax)
            Axis.contourf(Positions1,Positions2,Contourlevels,zdir=SliceAxes[Projection],offset=CostData[Axis3].min(),levels=SampledLevels,vmax=Vmax,alpha=0.2)
            
    Axis.plot([CostData[XAxis].min(),CostData[XAxis].min()],[y,y],[z,z],marker='o',color='Red')
    Axis.plot([x,x],[CostData[YAxis].max(),CostData[YAxis].max()],[z,z],marker='o',color='Red')
    Axis.plot([x,x],[y,y],[CostData[ZAxis].min(),CostData[ZAxis].min()],marker='o',color='Red')
    Axis.plot([x],[y],[z], marker='o',color='Black')

    # Plot settings
    Axis.set_xlim(CostData[XAxis].min(), CostData[XAxis].max())
    Axis.set_ylim(CostData[YAxis].min(), CostData[YAxis].max())
    Axis.set_zlim(CostData[ZAxis].min(), CostData[ZAxis].max())
    Axis.set_xlabel(XAxis+Units[0])
    Axis.set_ylabel(YAxis+Units[1])
    Axis.set_zlabel(ZAxis+Units[2])
    
    # make the panes transparent
    Axis.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    Axis.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    Axis.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    # make the grid lines transparent
    Axis.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    Axis.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    Axis.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
    
    plt.savefig('Plots/' + Tissue + 'MinPointProjection3D.png')
    
    plt.show()


Xmin, Xmax = CostData['Lambda'].min(), CostData['Lambda'].max()
Xstep = round((Xmax-Xmin)/(len(CostData['Lambda'].unique())-1),2)
Ymin, Ymax = CostData['Mu'].min(), CostData['Mu'].max()
Ystep = round((Ymax-Ymin)/(len(CostData['Mu'].unique())-1),2)
Zmin, Zmax = CostData['Alpha'].min(), CostData['Alpha'].max()
Zstep = round((Zmax-Zmin)/(len(CostData['Alpha'].unique())-1),0)

    
interactive_plot = interactive(f, x=(Xmin, Xmax, Xstep), y=(Ymin, Ymax, Ystep), z=(Zmin, Zmax, Zstep))
interactive_plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.4, description='x', max=2.2, min=0.2), FloatSlider(value=0.3, descri…

In [15]:
%matplotlib widget

AxisNames = ['Lambda', 'Mu', 'Alpha']
Units = [' (-)', ' (kPa)', ' (-)']

plt.rc('figure', figsize=[12,4])
Figure = plt.figure()

MinPoint = CostData['TotalCostNormMax'].idxmin()
    
def f(CompressionWeight=1, TensileWeight=1, SimpleShearWeight=1):
    
    CostData['CustomCost'] = CompressionWeight * CostData['CompressionCostNormMax'] + TensileWeight * CostData['TensileCostNormMax'] + SimpleShearWeight * CostData['SimpleShearCostNormMax']
    
    plt.clf()
    
    for Projection in range(len(AxisNames)):

        # Filter data to get the correct slice
        SliceAxis = CostData[AxisNames[Projection]].loc[MinPoint]
        Filter = CostData[AxisNames[Projection]] == SliceAxis
        Slice = CostData[Filter]

        # Data for plot
        XAxis = AxisNames[Projection-2]
        YAxis = AxisNames[Projection-1]

        XData = Slice[XAxis]
        YData = Slice[YAxis]
        Color = Slice['CustomCost'].values
        
        Table = Slice.pivot(XAxis,YAxis,'CustomCost')
        XPositions = Table.index.values
        YPositions = Table.columns.values
        Contourlevels = Table.transpose().values

        # Levels for contour
        SortedSlice = Slice.sort_values('CustomCost')
        SampledLevels = SortedSlice['CustomCost'][0::100].values
        Vmax = SampledLevels[-1]
        SampledLevels = np.append(SampledLevels,Slice['CustomCost'].max())

        # Plot
        Axes = Figure.add_subplot(1,3,Projection+1)
        Axes.scatter(XData,YData,c=Color,marker='.',s=10,vmax=Vmax)
        Axes.contour(XPositions,YPositions,Contourlevels,levels=SampledLevels,vmax=Vmax)
        CostContour = Axes.contourf(XPositions,YPositions,Contourlevels,levels=SampledLevels,vmax=Vmax,alpha=0.85)

        Axes.plot(CostData[XAxis][MinPoint], CostData[YAxis][MinPoint], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8)
        Axes.plot(OptiData[XAxis][len(OptiData)-1], OptiData[YAxis][len(OptiData)-1], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8)


        # Plot Settings
        Axes.set_xlabel(XAxis+Units[Projection-2])
        Axes.set_ylabel(YAxis+Units[Projection-1])

        ColorBar = Figure.colorbar(CostContour,ax=Axes, format='%0.2f')
        ColorBar.set_label('TotalCost (-)')

        if Projection+1 == len(AxisNames):
            plt.plot([], marker = '.', linestyle='none', color='y', markersize=5, label='Grid points')
            plt.plot([], marker = 'o', fillstyle='full', linestyle='none', color='k', markersize=8, label='Grid min cost\nCost: %.3f'%(CostData['CustomCost'].min()))
            plt.plot([], marker = 'o', fillstyle='none', linestyle='none', color='r', markersize=8, label='Optimization result\nCost: %.3f'%OptiData['TotalCostNormMax'][len(OptiData)-1])
            Figure.legend(loc='lower center', framealpha=1, ncol=3)
            plt.subplots_adjust(left=0.075, bottom=0.25, right=0.95, top=0.95, wspace=0.5, hspace=None)

    plt.show()
    
interactive_plot = interactive(f, CompressionWeight=(0.0, 3.0), TensileWeight=(0.0, 3.0), SimpleShearWeight=(0.0, 3.0))
display(interactive_plot)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=1.0, description='CompressionWeight', max=3.0), FloatSlider(value=1.0,…